In [8]:
#%% Imports
from serial import Serial
from PyQt5.QtWidgets import QMessageBox
from PyQt5.QtCore import QSettings

from PyQt5 import QtCore
import time


In [20]:
#%% Open port and load config file
portA='com5' 
mysA=Serial()
confMot=QSettings('configMoteurA2V.ini', QSettings.IniFormat) # motor configuration  files    

In [21]:
#%% Send and Receive data    
def sendCommand(instruction, mii, values_list):
    """
    Envoyer une commande au controleur
    'ROR':1, 'ROL':2, 'MST':3, 'MVP':4, 'SAP':5, 'GAP':6,
            'STAP':7, 'RSAP':8, 'SGP':9, 'GGP':10, 'RFS':13, 'SIO':14, 'GIO':15, 'WAIT':27, 'STOP':28,
                'SCO':30, 'GCO':31, 'CCO':32, 'VER':136, 'RST':255}
    """
    
    if len(values_list) > 4:
        print ("Command error: "+ str(values_list).encode('hex'))
        
    cmd = bytearray([0x01, instruction, instr_type, mii, 0x00, 0x00, 0x00, 0x00, 0x00])
    values_list.reverse()
    ii = 7
    for vii in values_list:
        cmd[ii] =( vii)
        ii = ii-1

        cmd[8] = sum(cmd[0:8])&0xff

        mysA.write(cmd)
        time.sleep(0.02)
        out = mysA.read(9)
        time.sleep(0.02)
        return bytearray(out)

        

def Format(value):
    """
    cmd to hex
    """
    return [(value>>24), ((value>>16)&0xff), ((value>>8)&0xff), (value&0xff)]


In [22]:
def connect(_port):
    """
    ouverture du port A
    """
    mysA.port=_port
    mysA.timeout=1
    mysA.Baudrate=9600
    if mysA.is_open==False:
        mysA.open()
    else:
        mysA.close()
        time.sleep(0.1)
        mysA.open()
    if mysA.is_open==True:
        print ('TMCM 1 connected on port :',_port)
    else: print('TMCM 1 on port',_port,' not connected')
        
def stopConnection():
    if mysA.is_open==True:
        try :
            mysA.close()
        except:
            print('error closing', _port)
            pass
    print ("TMCM disconnected")

In [29]:
   
def ini(motor=None):
    """ Motor init
    """
    vMax=int(confA2V.value(motor+'/vMax'))#1142
    iMax=int(confA2V.value(motor+'/iMax'))
    iStby=int(confA2V.value(motor+'/iStby'))
    accMax=int(confA2V.value(motor+'/accMax'))
    ustepRes=int(confA2V.value(motor+'/ustepRes')) # step resolution (0-8)
    
    cmd=5 # Set Axis parameter 
    Type=6 # max current
    value = Format(iMax)
    out = sendCommand(cmd,Type,Mot,value,rack)

    cmd=5 # Set Axis parameter 
    Type=7 # standby current
    value = Format(iStby)
    out = sendCommand(cmd,Type,Mot,value,rack)

    cmd=5 # Set Axis parameter 
    Type=4 # MAx speed
    value =Format(vMax)
    #print value
    out = sendCommand(cmd,Type,Mot,value,rack)

    cmd=5 # Set Axis parameter 
    Type=4 # MAx acc
    value =Format(accMax)
    #print value
    out = sendCommand(cmd,Type,Mot,value,rack)

    cmd=5 # Set Axis parameter 
    Type=140 # set step resolution
    value=Format(ustepRes) # entre 0 et 8 = entre 1 et 256
    out = sendCommand(cmd,Type,Mot,value,rack)
    #print (" motor A2V inititalisation :  ",motor)

In [30]:
stopConnection()
connect('com5')
ini('Generic')
stopConnection()

TMCM disconnected
TMCM 1 connected on port : com5


NameError: name 'Mot' is not defined